In [111]:
import re
import json
import pandas as pd
import random
pd.set_option('display.max_colwidth', None)

file_path = 'pre_dialogues.json'
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

In [112]:
df = pd.DataFrame(data)

In [113]:
df.head()

,prompt,response
0,카즈마 : ……………………안 오네……,메구밍 : 모집을 보고 왔습니다만. 면접은 여기서 하는 것인가?
1,메구밍 : 모집을 보고 왔습니다만. 면접은 여기서 하는 것인가?,"메구밍 : 내 이름은 메구밍. 아크 위저드를 하고 있으며 최강의 공격 마법, 폭렬마법을 다루는 자."
2,카즈마 : …………놀리러 온 건가?,"메구밍 : 아, 아니거든!"
3,"아쿠아 : ……그 붉은 눈동자. 너, 혹시 홍마족이야?","메구밍 : 그 말대로. 나는 홍마족 제일의 마법사 메구밍. ……저기, 뻔뻔한 부탁입니다만, 가능하면 뭔가 먹여주실 수 있으신가요……."
4,"아쿠아 : ……그러니까, 카즈마에게 설명하자면, 그들 홍마족은 타고난 지력과 강한 마력을 가지고 대부분이 강한 마법사의 소질을 갖고 있어. ……그리고 다들 별난 이름을 갖고 있어.",메구밍 : 별난 이름이라니 실례네요. 저로부터 보자면 다른 사람들의 이름이 훨씬 별나다고 생각합니다.


In [114]:
df['prompt'] = df['prompt'].apply(lambda x: re.sub(r'\.{3,}', '...', x.replace('…', '.')))
df['prompt'] = df['prompt'].apply(lambda x : x.replace('─', '').replace('―', '').replace('-', ''))
df['prompt'] = df['prompt'].apply(lambda x : x.replace('『', '').replace('』', ''))
df['prompt'] = df['prompt'].apply(lambda x : x.replace('【', '\"').replace('】', '\"'))
df['prompt'] = df['prompt'].apply(lambda x : x.replace('「', '\"').replace('」', '\"'))
df['prompt'] = df['prompt'].apply(lambda x : x.replace('ㆍ', ' ').replace('(爆裂道)', ''))
df['prompt'] = df['prompt'].apply(lambda x : x.replace('ㄸ', '또'))

df['response'] = df['response'].apply(lambda x: re.sub(r'\.{3,}', '...', x.replace('…', '.')))
df['response'] = df['response'].apply(lambda x : x.replace('─', '').replace('―', '').replace('-', ''))
df['response'] = df['response'].apply(lambda x : x.replace('『', '').replace('』', ''))
df['response'] = df['response'].apply(lambda x : x.replace('【', '\"').replace('】', '\"'))
df['response'] = df['response'].apply(lambda x : x.replace('「', '\"').replace('」', '\"'))
df['response'] = df['response'].apply(lambda x : x.replace('ㆍ', ' ').replace('(爆裂道)', ''))
df['response'] = df['response'].apply(lambda x : x.replace('ㄸ', '또'))

In [115]:
df['response_length'] = df.response.apply(lambda x : len(x))
df = df.loc[df.response_length >= 3]

In [116]:
with open('dialogues.json', "r", encoding="utf-8") as f:
        data = json.load(f)

In [117]:
data = pd.DataFrame(data[:940])
data.columns = ['prompt', 'response']
data.prompt = data.prompt.apply(lambda x : "사용자 : " + x)
data.response = data.response.apply(lambda x : "메구밍 : " + x)

new_df = pd.concat([df[['prompt', 'response']], data], axis = 0)

In [118]:
def split_and_shuffle_sentences_with_offset(text):
    # 특수 구두점 패턴 및 일반 구두점 패턴 구분
    pattern = r'([.]{3}|[?!]{2}|[.?!])'
    sentences = re.split(pattern, text)
    
    # 문장 및 구두점을 한 쌍으로 결합
    sentences = [''.join(x) for x in zip(sentences[0::2], sentences[1::2])]
    
    # 앞부분 6개 문장 유지
    preserved_sentences = sentences[:6]
    
    # 특수 구두점 패턴이 있는 문장은 그대로 두고 나머지 문장들만 섞음 (6번째 이후)
    shuffled_sentences = []
    temp_sentence = []
    for sentence in sentences[6:]:
        if '...' in sentence or '?!' in sentence:
            if temp_sentence:
                random.shuffle(temp_sentence)  # 일반 문장 섞기
                shuffled_sentences.extend(temp_sentence)
                temp_sentence = []
            shuffled_sentences.append(sentence)
        else:
            temp_sentence.append(sentence)
    
    # 남아있는 일반 문장 섞기
    if temp_sentence:
        random.shuffle(temp_sentence)
        shuffled_sentences.extend(temp_sentence)
    
    # 처음 6개 문장과 섞인 문장 결합
    return ' '.join(preserved_sentences + shuffled_sentences)

# 증강된 텍스트를 적용하고 prompt와 response 둘 다 유지
augmented_texts_with_offset = new_df['response'].apply(split_and_shuffle_sentences_with_offset)

# 증강된 response를 새로운 데이터프레임으로 추가
augmented_df_with_offset = pd.concat([new_df, pd.DataFrame({'prompt': new_df['prompt'], 'response': augmented_texts_with_offset})], ignore_index=True)

# 중복 제거 (가장 처음 값만 남기기)
augmented_df_with_offset = augmented_df_with_offset.drop_duplicates(subset=['prompt', 'response'], keep='first')
augmented_df_with_offset = augmented_df_with_offset.loc[augmented_df_with_offset.response != '']
augmented_df_with_offset = augmented_df_with_offset.reset_index(drop = True)

In [122]:
augmented_df_with_offset.prompt.apply(lambda x : len(x)).max()

np.int64(414)

In [119]:
# DataFrame을 prompt, response 형식의 JSON 파일로 저장하는 코드
json_data = augmented_df_with_offset[['prompt', 'response']].to_dict(orient='records')

# JSON 파일로 저장
output_file_path = 'augmented_prompt_response.json'
with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

## 깃허브 데이터 한국어로

In [2]:
import pandas as pd

# Function to read a text file and convert it to a dataframe
def text_to_dataframe(file_path):
    # Read the file content
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.readlines()
    
    # Create a dataframe where each line of text is a row
    df = pd.DataFrame(content, columns=["text"])
    
    return df

# Example usage:
file_path = "konosuba.txt"  # Path to the file
df = text_to_dataframe(file_path)

In [12]:
from tqdm import tqdm
from deep_translator import GoogleTranslator
pd.options.display.max_colwidth = 1000
tqdm.pandas()

# GoogleTranslator 객체를 함수 외부에서 생성해 재사용
translator = GoogleTranslator(source='en', target='ko')

# GoogleTranslator를 사용하여 한국어 -> 영어 번역 함수 정의
def translate_text(text):
    return translator.translate(text)

# text 컬럼의 모든 행을 번역하여 translated_text 컬럼으로 할당
df['translated_text'] = df['text'].progress_apply(translate_text)

100%|██████████| 47573/47573 [11:05:02<00:00,  1.19it/s]   


In [29]:
df

,text,translated_text
0,"“Satou Kazuma-san, welcome to the afterlife. Unfortunately, you’ve died. It might’ve been short, but your life’s now over.”\n","“사토 카즈마 씨, 저승에 오신 것을 환영합니다. 불행히도 당신은 죽었습니다. 짧았을지 몰라도 당신의 삶은 이제 끝났습니다.”"
1,Someone suddenly spoke to me in a pure white room.\n,갑자기 새하얀 방에서 누군가가 나에게 말을 걸었습니다.
2,The sudden turn of events confused me.\n,갑작스러운 사건의 전개에 나는 혼란스러웠다.
3,"In the room was an office desk and a chair, and the one who announced that my life was over sat on said chair.\n","방 안에는 사무실 책상과 의자가 있었고, 내 인생이 끝났다고 선언한 사람이 그 의자에 앉아 있었습니다."
4,"If there was a goddess, she had to be it.\n","여신이 있다면, 그녀가 바로 그 여신이어야 했다."
...,...,...
47200,"“In the first place, Darkness, you are way too obsessed with slimy stuff! The other day too, you were gazing enviously at the piece of Tokoroten slime I got. You really shouldn’t play with food!”\n","""어차피 다크니스, 넌 끈적끈적한 것에 너무 집착하고 있어! 얼마 전 내가 가져온 토코로텐 슬라임을 부러워하며 바라보았잖아. 음식으로 장난치면 안 돼!"""
47201,“Even I wouldn’t do something like that! My body just reacted upon hearing the word ‘slime’…!”\n,“나도 그런 짓은 안 해! ‘슬라임’이라는 단어만 들어도 몸이 반응하잖아…!”
47202,"As the voices of these inept girls drifted through this calm afternoon, I enjoyed my mug of beer.\n","이 무능한 소녀들의 목소리가 고요한 오후에 흘러나오는 동안, 나는 맥주 한 잔을 즐겼다."
47203,"“Hey, Aqua! I was planning to take a bath after everyone was clean! Go out and buy some shampoo and soap!”\n","""야, 아쿠아! 다들 깨끗이 씻은 후에 목욕할 계획이었어! 나가서 샴푸랑 비누 사와!"""


In [21]:
df = df.dropna().reset_index(drop = True)

In [98]:
df['pre_translated_text'] = df.translated_text.replace('…', '..', regex = True).\
                                        replace('“', '\"', regex = True).replace('”', '\"', regex = True).\
                                        replace('‘', '\'', regex = True).replace('’', '\'', regex = True).\
                                        replace('─', '~', regex = True).replace('-', ' ', regex = True).\
                                        replace('–', '', regex = True).replace('ー', '', regex = True).\
                                        replace('―', '', regex = True).replace(';', '', regex = True).\
                                        replace(']', '', regex = True).replace('—', '', regex = True).\
                                        replace('ぁ', '아', regex = True).replace('ど', '', regex = True).\
                                        replace('×2', '', regex = True).replace('×3', '', regex = True).\
                                        replace('(聖器)', '', regex = True).replace('↓', '', regex = True).\
                                        replace('·', '', regex = True)

In [99]:
df.head()

,text,translated_text,pre_translated_text
0,"“Satou Kazuma-san, welcome to the afterlife. Unfortunately, you’ve died. It might’ve been short, but your life’s now over.”\n","“사토 카즈마 씨, 저승에 오신 것을 환영합니다. 불행히도 당신은 죽었습니다. 짧았을지 몰라도 당신의 삶은 이제 끝났습니다.”","""사토 카즈마 씨, 저승에 오신 것을 환영합니다. 불행히도 당신은 죽었습니다. 짧았을지 몰라도 당신의 삶은 이제 끝났습니다."""
1,Someone suddenly spoke to me in a pure white room.\n,갑자기 새하얀 방에서 누군가가 나에게 말을 걸었습니다.,갑자기 새하얀 방에서 누군가가 나에게 말을 걸었습니다.
2,The sudden turn of events confused me.\n,갑작스러운 사건의 전개에 나는 혼란스러웠다.,갑작스러운 사건의 전개에 나는 혼란스러웠다.
3,"In the room was an office desk and a chair, and the one who announced that my life was over sat on said chair.\n","방 안에는 사무실 책상과 의자가 있었고, 내 인생이 끝났다고 선언한 사람이 그 의자에 앉아 있었습니다.","방 안에는 사무실 책상과 의자가 있었고, 내 인생이 끝났다고 선언한 사람이 그 의자에 앉아 있었습니다."
4,"If there was a goddess, she had to be it.\n","여신이 있다면, 그녀가 바로 그 여신이어야 했다.","여신이 있다면, 그녀가 바로 그 여신이어야 했다."


In [105]:
pattern = r'[^\u200b-\u200d\ufeff가-힣a-zA-Z0-9 .,?!\'\"~*:%()&/]'
new_df = df[~df['pre_translated_text'].str.contains(pattern)]

In [113]:
# DataFrame의 translated_text 컬럼을 JSON 형식으로 저장하는 함수
def save_translated_text_to_json(df, output_filename):
    # translated_text 컬럼을 리스트로 추출
    translated_text_list = df['text'].tolist()

    # "메구밍: "을 문장 앞에 추가
    updated_translated_text_list = []
    for text in translated_text_list:
        # 대화 형식의 문자열만 "메구밍: "을 앞에 추가
        if text.startswith("\"") and text.endswith("\""):
            updated_translated_text_list.append(f'발화자 : {text}')
        else:
            updated_translated_text_list.append(text)

    # JSON 형식으로 변환 및 파일 저장
    with open(output_filename, 'w', encoding='utf-8') as json_file:
        json.dump(updated_translated_text_list, json_file, ensure_ascii=False, indent=4)

# 예시로 JSON 파일 저장
save_translated_text_to_json(new_df, 'text_output.json')

In [117]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("silk-road/ChatHaruhi_RolePlaying_qwen_7b", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("silk-road/ChatHaruhi_RolePlaying_qwen_7b", device_map="auto", trust_remote_code=True)
model = model.eval()


ImportError: Loading a GPTQ quantized model requires optimum (`pip install optimum`) and auto-gptq library (`pip install auto-gptq`)